In [3]:
import pandas as pd
import os

# 文件路径
basicincident_file = os.path.join('DATA_nfirs2000-2021/NFIRS2019/basicincident.txt')
processed_csv_file = os.path.join('morans_data/2019/basicincident_processed_2019.csv')

# 处理并转换日期时间格式的函数
def convert_datetime(dt_str):
    return pd.to_datetime(dt_str, format='%m%d%Y%H%M', errors='coerce')

# 处理basicincident.txt文件
def process_basicincident(file_path, output_file):
    chunk_size = 10000  # 每次读取的行数
    total_rows = 0  # 记录总行数
    ca_rows = 0  # 记录CA行数

    # 初始化输出文件
    first_chunk = True

    for chunk in pd.read_csv(file_path, delimiter='^', encoding='latin1', low_memory=False, chunksize=chunk_size):
        total_rows += len(chunk)
        # 过滤出加利福尼亚州的数据
        chunk_ca = chunk[chunk['STATE'] == 'CA']
        ca_rows += len(chunk_ca)

        # 转换日期和时间格式
        chunk_ca['ALARM'] = pd.to_datetime(chunk_ca['ALARM'], format='%m%d%Y%H%M', errors='coerce')
        chunk_ca['ARRIVAL'] = pd.to_datetime(chunk_ca['ARRIVAL'], format='%m%d%Y%H%M', errors='coerce')
        chunk_ca['LU_CLEAR'] = pd.to_datetime(chunk_ca['LU_CLEAR'], format='%m%d%Y%H%M', errors='coerce')

        # 丢弃ALARM和ARRIVAL中为空的行
        chunk_ca = chunk_ca.dropna(subset=['ALARM', 'ARRIVAL'])

        # 确保列转换为日期时间类型
        chunk_ca = chunk_ca[chunk_ca['ALARM'].notnull() & chunk_ca['ARRIVAL'].notnull()]
        chunk_ca['ALARM'] = pd.to_datetime(chunk_ca['ALARM'], errors='coerce')
        chunk_ca['ARRIVAL'] = pd.to_datetime(chunk_ca['ARRIVAL'], errors='coerce')
        chunk_ca['LU_CLEAR'] = pd.to_datetime(chunk_ca['LU_CLEAR'], errors='coerce')

        # 计算响应时间和解决火灾的时间
        chunk_ca['RESPONSE_TIME'] = (chunk_ca['ARRIVAL'] - chunk_ca['ALARM']).dt.total_seconds() / 60  # 以分钟为单位
        chunk_ca['FIRE_CLEARANCE_TIME'] = (chunk_ca['LU_CLEAR'] - chunk_ca['ARRIVAL']).dt.total_seconds() / 60  # 以分钟为单位

        # 写入CSV文件
        if first_chunk:
            chunk_ca.to_csv(output_file, index=False)
            first_chunk = False
        else:
            chunk_ca.to_csv(output_file, mode='a', header=False, index=False)

    print(f"Processed data saved to {output_file}")
    print(f"Total rows: {total_rows}, CA rows: {ca_rows}")

# 运行处理函数
process_basicincident(basicincident_file, processed_csv_file)


/tmp/ipykernel_23244/3293934736.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk_ca['ALARM'] = pd.to_datetime(chunk_ca['ALARM'], format='%m%d%Y%H%M', errors='coerce')
/tmp/ipykernel_23244/3293934736.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk_ca['ARRIVAL'] = pd.to_datetime(chunk_ca['ARRIVAL'], format='%m%d%Y%H%M', errors='coerce')
/tmp/ipykernel_23244/3293934736.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

Processed data saved to morans_data/2019/basicincident_processed_2019.csv
Total rows: 28534829, CA rows: 3359106


In [5]:
import pandas as pd
import geopandas as gpd

# 文件路径
processed_csv_file = 'morans_data/2019/basicincident_processed_2019.csv'
shapefile_path = 'california_shapefile/FireDistricts24_1.shp'
output_file = 'morans_data/2019/filtered_fire_data_2019.csv'

# 读取 shapefile 数据
gdf = gpd.read_file(shapefile_path)

# 确保 FDID 列为字符串类型
gdf['FDID'] = gdf['FDID'].astype(str)

# 空列表存储分块处理后的数据
chunks = []
chunk_size = 10000  # 每次处理的行数

# 分块读取火灾数据并过滤加利福尼亚州的数据
dtypes = {
    'FDID': str,
    'STATE': str,
    'RESPONSE_TIME': float,
    'FIRE_CLEARANCE_TIME': float,
    'ALARM': str,
    'ARRIVAL': str,
    'LU_CLEAR': str
}

for chunk in pd.read_csv(processed_csv_file, chunksize=chunk_size, dtype=dtypes, low_memory=False):
    chunk_ca = chunk[chunk['STATE'] == 'CA']
    chunks.append(chunk_ca[['FDID', 'RESPONSE_TIME', 'FIRE_CLEARANCE_TIME']])

# 合并所有分块
fire_data_ca = pd.concat(chunks)

# 保存处理后的数据
fire_data_ca.to_csv(output_file, index=False)
print(f"Filtered data saved to {output_file}")


Filtered data saved to morans_data/2019/filtered_fire_data_2019.csv


In [1]:
import dask.dataframe as dd
import geopandas as gpd
from libpysal.weights import Queen
from esda.moran import Moran
from scipy.sparse import csr_matrix
import numpy as np

# 文件路径
filtered_data_file = 'morans_data/2019/filtered_fire_data_2019.csv'
shapefile_path = 'california_shapefile/FireDistricts24_1.shp'

# 读取过滤后的火灾数据
fire_data_ca = dd.read_csv(filtered_data_file)

# 读取 shapefile 数据
gdf = gpd.read_file(shapefile_path)

# 确保火灾数据中的 FDID 列为字符串类型
fire_data_ca['FDID'] = fire_data_ca['FDID'].astype(str)
gdf['FDID'] = gdf['FDID'].astype(str)

# 将 fire_data_ca 转换为 pandas DataFrame
fire_data_ca = fire_data_ca.compute()

# 合并数据并仅保留必要的列
merged = gdf[['FDID', 'geometry']].merge(fire_data_ca[['FDID', 'RESPONSE_TIME', 'FIRE_CLEARANCE_TIME']], on='FDID', how='inner')

# 检查合并后的数据大小
print(f"Size of merged data: {merged.shape}")

# 检查合并后的数据
print(merged.head())


/opt/conda/lib/python3.11/site-packages/libpysal/cg/alpha_shapes.py:38: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/opt/conda/lib/python3.11/site-packages/libpysal/cg/alpha_shapes.py:164: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/opt/conda/lib/python3.11/site-packages/libpysal/cg/alpha_shapes.py:198: NumbaDeprecationWarning: The 'nopython' keyw

Size of merged data: (2613313, 4)
    FDID                                           geometry  RESPONSE_TIME  \
0  10005  MULTIPOLYGON (((34471.405 -135546.341, 34566.2...            3.0   
1  10005  MULTIPOLYGON (((34471.405 -135546.341, 34566.2...            4.0   
2  10005  MULTIPOLYGON (((34471.405 -135546.341, 34566.2...            4.0   
3  10005  MULTIPOLYGON (((34471.405 -135546.341, 34566.2...            4.0   
4  10005  MULTIPOLYGON (((34471.405 -135546.341, 34566.2...            2.0   

   FIRE_CLEARANCE_TIME  
0                  9.0  
1                  9.0  
2                  9.0  
3                 14.0  
4                  8.0  


In [2]:
import geopandas as gpd

# 读取 Shapefile 数据
shapefile_path = 'california_shapefile/FireDistricts24_1.shp'
gdf = gpd.read_file(shapefile_path)

# 确保 GeoDataFrame 中的 FDID 列为字符串类型
gdf['FDID'] = gdf['FDID'].astype(str)

# 检查读取的数据
print(gdf.head())


    County   FDID MACSID                               Name  \
0  ALAMEDA    nan    NaN               CAMP PARKS FIRE DEPT   
1  ALAMEDA    nan    NaN  FAIRVIEW FIRE PROTECTION DISTRICT   
2  ALAMEDA  01005    ALA                         ALAMEDA FD   
3  ALAMEDA  01008    ACF                  ALAMEDA COUNTY FD   
4  ALAMEDA  01010    ALB                  CITY OF ALBANY FD   

              Address     City    Zip            FireChief           Phone  \
0                 NaN      NaN    NaN                  NaN             NaN   
1                 NaN      NaN    NaN                  NaN  (510) 583-4940   
2        1300 PARK ST  ALAMEDA  94501        NICHOLAS LUBY  (510) 337-2100   
3      6363 CLARK AVE   DUBLIN  94568  WILLIAM L. MCDONALD  (925) 833-3473   
4  1000 SAN PABLO AVE   ALBANY  94706          JAMES BOITO  (510) 528-5770   

  Notes  LastUpdate                                            Website  \
0   NaN  2018-06-21                                             <Null>   
1   

In [3]:
import pandas as pd

# 定义一个处理数据块的函数
def process_chunk(chunk):
    chunk['FDID'] = chunk['FDID'].astype(str)

    # 仅保留必要的列
    chunk = chunk[['FDID', 'RESPONSE_TIME', 'FIRE_CLEARANCE_TIME']]
    
    # 合并数据
    merged_chunk = gdf[['FDID', 'geometry']].merge(chunk, on='FDID', how='inner')
    return merged_chunk


In [3]:
import dask.dataframe as dd
import geopandas as gpd
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

# 文件路径
filtered_data_file = 'morans_data/2019/filtered_fire_data_2019.csv'
shapefile_path = 'california_shapefile/FireDistricts24_1.shp'

# 读取 Shapefile 数据
gdf = gpd.read_file(shapefile_path)
gdf['FDID'] = gdf['FDID'].astype(str)

# 使用 Dask 读取过滤后的火灾数据
fire_data_ca = dd.read_csv(filtered_data_file, usecols=['FDID', 'RESPONSE_TIME', 'FIRE_CLEARANCE_TIME'])
fire_data_ca['FDID'] = fire_data_ca['FDID'].astype(str)

# 定义处理数据块的函数
def process_chunk(chunk):
    chunk['FDID'] = chunk['FDID'].astype(str)
    merged_chunk = gdf[['FDID', 'geometry']].merge(chunk, on='FDID', how='inner')
    return merged_chunk

# 创建元数据
meta = gpd.GeoDataFrame({
    'FDID': pd.Series(dtype='str'),
    'geometry': gpd.GeoSeries([], crs=gdf.crs),
    'RESPONSE_TIME': pd.Series(dtype='float64'),
    'FIRE_CLEARANCE_TIME': pd.Series(dtype='float64')
})

# 使用 Dask 进行分块处理
fire_data_ca = fire_data_ca.map_partitions(process_chunk, meta=meta)

# 将 Dask DataFrame 转换为 Pandas DataFrame 以进行后续处理
fire_data_pd = fire_data_ca.compute()

# 删除包含缺失值的样本
fire_data_pd = fire_data_pd.dropna(subset=['RESPONSE_TIME', 'FIRE_CLEARANCE_TIME'])

# 标准化数据
scaler = StandardScaler()
data_scaled = scaler.fit_transform(fire_data_pd[['RESPONSE_TIME', 'FIRE_CLEARANCE_TIME']])

# 使用 Elbow 方法选择 K 值
inertia = []
K_range = range(1, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_scaled)
    inertia.append(kmeans.inertia_)

# 可视化 Elbow 方法
plt.figure(figsize=(8, 4))
plt.plot(K_range, inertia, marker='o')
plt.xlabel('Number of clusters, K')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal K')
plt.show()

# 应用 K-means 聚类 (假设选择的 K 值为 3)
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(data_scaled)

# 将聚类结果添加到原始数据中
fire_data_pd['Cluster'] = clusters

# 检查聚类结果
print(fire_data_pd.head())

# 可视化聚类结果
plt.figure(figsize=(10, 6))
sns.scatterplot(x=fire_data_pd['RESPONSE_TIME'], y=fire_data_pd['FIRE_CLEARANCE_TIME'], hue=fire_data_pd['Cluster'], palette='viridis')
plt.title('K-means Clustering of Response Time and Fire Clearance Time (2019)')
plt.xlabel('Response Time (minutes)')
plt.ylabel('Fire Clearance Time (minutes)')
plt.legend(title='Cluster')
plt.show()


ValueError: Metadata inference failed in `_to_string_dtype`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
TypeError("Cannot interpret '<geopandas.array.GeometryDtype object at 0xffff5d502dd0>' as a data type")

Traceback:
---------
  File "/opt/conda/lib/python3.11/site-packages/dask/dataframe/utils.py", line 193, in raise_on_meta_error
    yield
  File "/opt/conda/lib/python3.11/site-packages/dask/dataframe/core.py", line 6897, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/dask/dataframe/core.py", line 6877, in _extract_meta
    return tuple(_extract_meta(_x, nonempty) for _x in x)
           ^^^^^^^^^^^^^^^^^^^^^

In [ ]:
#=================response time==================

In [ ]:
import dask.dataframe as dd

# 使用Dask读取合并后的数据
merged_data_file = 'morans_data/2019/merged_fire_data_2019.csv'
merged = dd.read_csv(merged_data_file)

# 确保Dask dataframe中FDID是字符串类型
merged['FDID'] = merged['FDID'].astype(str)

# 显示前几行，确保数据正确读取
print(merged.head())


In [ ]:
# 为 FDID 生成一个索引
fdid_unique = merged['FDID'].unique().compute()
fdid_index = {fdid: idx for idx, fdid in enumerate(fdid_unique)}
merged['FDID_index'] = merged['FDID'].map(fdid_index)

# 显示前几行，确保索引正确生成
print(merged.head())


In [ ]:
import numpy as np
from libpysal.weights import DistanceBand
from esda.moran import Moran

# 定义计算 Moran's I 的函数
def compute_moran(chunk):
    fdid_index_values = chunk['FDID_index'].values
    y_response = chunk['RESPONSE_TIME'].values
    
    # 创建权重矩阵
    coords = np.array(list(enumerate(fdid_index_values)))
    w = DistanceBand(coords, threshold=1.5, binary=True, silence_warnings=True)
    
    moran_response = Moran(y_response, w)
    return moran_response.I, moran_response.p_norm


In [ ]:
# 分块计算 Moran's I
results = merged.map_partitions(compute_moran, meta=('I', 'f8')).compute()

# 显示计算结果
print(results)


In [ ]:
import pandas as pd
# 将结果转换为 DataFrame
results_df = pd.DataFrame(results.tolist(), columns=['Moran_I', 'p_value'])

# 添加 FDID 列
results_df['FDID'] = merged['FDID'].unique().compute()

# 保存结果到 CSV 文件
results_df.to_csv('morans_data/2019/morans_i_results2019_rt.csv', index=False)

In [ ]:
# 计算总体的 Moran's I 平均值
average_moran_i = np.mean([res[0] for res in results])
average_p_value = np.mean([res[1] for res in results])

# 输出结果
print(f"Average Response Time Moran's I: {average_moran_i}")
print(f"Average Response Time P-value: {average_p_value}")

if average_p_value < 0.05:
    print("存在显著的火灾响应时间空间自相关")
else:
    print("没有显著的火灾响应时间空间自相关")


In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# 读取 Shapefile 文件
shapefile_path = 'california_shapefile/FireDistricts24_1.shp'
gdf = gpd.read_file(shapefile_path)

# 确保 GeoDataFrame 中 FDID 是字符串类型
gdf['FDID'] = gdf['FDID'].astype(str)

# 检查并设置 CRS 为 EPSG:4326
gdf = gdf.to_crs(epsg=4326)

# 读取保存的 Moran's I 结果
results_df = pd.read_csv('morans_data/2019/morans_i_results2019_rt.csv')

# 过滤掉包含空白 FDID 的行
results_df = results_df[results_df['FDID'].notna()]

# 将 FDID 转换为整数类型并再转换为字符串类型
results_df['FDID'] = results_df['FDID'].astype(int).astype(str)

# 检查合并前的数据
print(results_df.head())
print(gdf.head())

# 将 Moran's I 结果合并到 GeoDataFrame 中
gdf = gdf.merge(results_df, on='FDID', how='left')

# 检查合并后的数据
print(gdf.head())

# 绘制地图，展示 Moran's I 结果，设置缺失数据的颜色
fig, ax = plt.subplots(1, 1, figsize=(15, 15))
gdf.plot(column='Moran_I', cmap='coolwarm', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True, 
         missing_kwds={"color": "lightgrey", "label": "No data"})
ax.set_title('Moran\'s I of Fire RESPONSE Time by FDID')
plt.show()


In [ ]:
#============='FIRE_CLEARANCE_TIME'===============

In [ ]:
import dask.dataframe as dd

# 使用Dask读取合并后的数据
merged_data_file = 'morans_data/2019/merged_fire_data_2019.csv'
merged = dd.read_csv(merged_data_file)

# 确保Dask dataframe中FDID是字符串类型
merged['FDID'] = merged['FDID'].astype(str)

# 显示前几行，确保数据正确读取
print(merged.head())


In [ ]:
# 为 FDID 生成一个索引
fdid_unique = merged['FDID'].unique().compute()
fdid_index = {fdid: idx for idx, fdid in enumerate(fdid_unique)}
merged['FDID_index'] = merged['FDID'].map(fdid_index)

# 显示前几行，确保索引正确生成
print(merged.head())


In [ ]:
import numpy as np
from libpysal.weights import DistanceBand
from esda.moran import Moran

# 定义计算 Moran's I 的函数
def compute_moran(chunk):
    fdid_index_values = chunk['FDID_index'].values
    y_response = chunk['FIRE_CLEARANCE_TIME'].values
    
    # 创建权重矩阵
    coords = np.array(list(enumerate(fdid_index_values)))
    w = DistanceBand(coords, threshold=1.5, binary=True, silence_warnings=True)
    
    moran_response = Moran(y_response, w)
    return moran_response.I, moran_response.p_norm


In [ ]:
# 分块计算 Moran's I
results = merged.map_partitions(compute_moran, meta=('I', 'f8')).compute()

# 显示计算结果
print(results)


In [ ]:
import pandas as pd
# 将结果转换为 DataFrame
results_df = pd.DataFrame(results.tolist(), columns=['Moran_I', 'p_value'])

# 添加 FDID 列
results_df['FDID'] = merged['FDID'].unique().compute()

# 保存结果到 CSV 文件
results_df.to_csv('morans_data/2019/morans_i_results2019_clear.csv', index=False)

In [ ]:
# 过滤掉包含 nan 的结果
filtered_results = [res for res in results if not np.isnan(res[0]) and not np.isnan(res[1])]

# 检查是否有有效的结果
if filtered_results:
    # 计算总体的 Moran's I 平均值
    average_moran_i = np.mean([res[0] for res in filtered_results])
    average_p_value = np.mean([res[1] for res in filtered_results])

    # 输出结果
    print(f"Average Fire Clearance Time Moran's I: {average_moran_i}")
    print(f"Average Fire Clearance Time P-value: {average_p_value}")

    if average_p_value < 0.05:
        print("存在显著的火灾清除时间空间自相关")
    else:
        print("没有显著的火灾清除时间空间自相关")
else:
    print("所有分区的计算结果都包含 nan 值，无法计算总体的 Moran's I 平均值。")


In [ ]:
import geopandas as gpd

# 读取Shapefile文件
shapefile_path = 'california_shapefile/FireDistricts24_1.shp'
gdf = gpd.read_file(shapefile_path)

# 确保GeoDataFrame中FDID是字符串类型
gdf['FDID'] = gdf['FDID'].astype(str)

# 显示前几行，确保数据正确读取
print(gdf.head())


In [ ]:
import dask.dataframe as dd
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from libpysal.weights import DistanceBand
from esda.moran import Moran

# 使用 Dask 读取合并后的数据，这里包含response time
merged_data_file = 'morans_data/2019/merged_fire_data_2019.csv'
merged = dd.read_csv(merged_data_file)

# 确保 Dask DataFrame 中 FDID 是字符串类型
merged['FDID'] = merged['FDID'].astype(str)

# 读取 Shapefile 文件
shapefile_path = 'california_shapefile/FireDistricts24_1.shp'
gdf = gpd.read_file(shapefile_path)

# 确保 GeoDataFrame 中 FDID 是字符串类型
gdf['FDID'] = gdf['FDID'].astype(str)

# 检查并设置 CRS 为 EPSG:4326
gdf = gdf.to_crs(epsg=4326)


In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# 读取 Shapefile 文件
shapefile_path = 'california_shapefile/FireDistricts24_1.shp'
gdf = gpd.read_file(shapefile_path)

# 确保 GeoDataFrame 中 FDID 是字符串类型
gdf['FDID'] = gdf['FDID'].astype(str)

# 检查并设置 CRS 为 EPSG:4326
gdf = gdf.to_crs(epsg=4326)

# 读取保存的 Moran's I 结果
results_df = pd.read_csv('morans_data/2019/morans_i_results2019_clear.csv')

# 过滤掉包含空白 FDID 的行
results_df = results_df[results_df['FDID'].notna()]

# 将 FDID 转换为整数类型并再转换为字符串类型
results_df['FDID'] = results_df['FDID'].astype(int).astype(str)

# 检查合并前的数据
print(results_df.head())
print(gdf.head())

# 将 Moran's I 结果合并到 GeoDataFrame 中
gdf = gdf.merge(results_df, on='FDID', how='left')

# 检查合并后的数据
print(gdf.head())

# 绘制地图，展示 Moran's I 结果，设置缺失数据的颜色
fig, ax = plt.subplots(1, 1, figsize=(15, 15))
gdf.plot(column='Moran_I', cmap='coolwarm', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True, 
         missing_kwds={"color": "lightgrey", "label": "No data"})
ax.set_title('Moran\'s I of Fire Clearance Time by FDID')
plt.show()


In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# 读取 Shapefile 文件
shapefile_path = 'california_shapefile/FireDistricts24_1.shp'
gdf = gpd.read_file(shapefile_path)

# 确保 GeoDataFrame 中 FDID 是字符串类型
gdf['FDID'] = gdf['FDID'].astype(str)

# 检查并设置 CRS 为 EPSG:4326
gdf = gdf.to_crs(epsg=4326)

# 读取保存的 Moran's I 结果
results_df = pd.read_csv('morans_data/2019/morans_i_results2019_clear.csv')

# 过滤掉包含空白 FDID 的行
results_df = results_df[results_df['FDID'].notna()]

# 将 FDID 转换为整数类型并再转换为字符串类型
results_df['FDID'] = results_df['FDID'].astype(int).astype(str)

# 检查合并前的数据
print(results_df.head())
print(gdf.head())

# 将 Moran's I 结果合并到 GeoDataFrame 中
gdf = gdf.merge(results_df, on='FDID', how='left')

# 检查合并后的数据
print(gdf.head())

# 绘制地图，展示 Moran's I 结果，设置缺失数据的颜色
fig, ax = plt.subplots(1, 1, figsize=(15, 15))
gdf.plot(column='Moran_I', cmap='coolwarm', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True, 
         missing_kwds={"color": "lightgrey", "label": "No data"})
ax.set_title('Moran\'s I of Fire Clearance Time by FDID')
plt.show()
